In [1]:
#!pip install open3d
#!conda install -c open3d-admin open3d

In [2]:
import open3d as o3d
import numpy as np

In [106]:
#pointCloudMatrix = [[0.0, 0, 0],[0, 0, 1],[0, 1, 0],[0, 1, 1],[1, 0, 0],[1, 0, 1],[1, 1, 0],[1, 1, 1]];

#pointCloudMatrix = np.array([[0,0,0]])
#for x in range(51):
   # for y in range(51):
       # for z in range(51):
          #  if (abs(x/50) == 1 or abs(y/50) == 1 or abs(z/50)==1 or abs(x) == 0 or abs(y) == 0 or abs(z)==0):
              #  pointCloudMatrix = np.concatenate((pointCloudMatrix,[[x/3,y/3,z/3]]),axis=0)
#pointCloudMatrix = np.array(np.loadtxt('np_array.txt'))                
pointCloudMatrix = np.array(np.loadtxt('decimated_hq_room_half.txt'))            


In [107]:
pointCloudMatrix.shape

(23065, 6)

In [108]:
pointCloudMatrix= pointCloudMatrix[:,0:3]
pointCloudMatrix.shape

(23065, 3)

In [109]:
def simplistic_mesh_creator_voxel(pointCloudMatrix):
     #takes a numpy matrix of size [x,3] of points 
     #and uses open3d inorder to create a basic voxel representation of the points
    
    o3dPointCloud = o3d.geometry.PointCloud() #Create pointcloud object
    o3dPointCloud.points = o3d.utility.Vector3dVector(pointCloudMatrix) #Populate with points from numpy matrix
    o3dPointCloud.scale(1 / np.max(o3dPointCloud.get_max_bound() - o3dPointCloud.get_min_bound()),
          center=o3dPointCloud.get_center()) #fix the scale
    #o3dPointCloud.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
    #voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(o3dPointCloud,
                                                            #voxel_size=0.05)
        
    v_size = round(max(o3dPointCloud.get_max_bound()-o3dPointCloud.get_min_bound())*0.005,4) #find Side of each voxel
    voxel_grid=o3d.geometry.VoxelGrid.create_from_point_cloud(o3dPointCloud,voxel_size=v_size)
   # o3d.visualization.draw_geometries([voxel_grid]) #visualization
    
    voxels=voxel_grid.get_voxels()
    vox_mesh = o3d.geometry.TriangleMesh()
    #Generate Meshes from the voxels
    for v in voxels:
        cube = o3d.geometry.TriangleMesh.create_box(width=1, height=1,depth=1)
        cube.paint_uniform_color(v.color)
        cube.translate(v.grid_index, relative=False)
        vox_mesh+=cube
    
    vox_mesh.translate([0.5,0.5,0.5], relative=True)
    vox_mesh.scale(v_size, [0,0,0])
    vox_mesh.translate(voxel_grid.origin, relative=True)
    vox_mesh.merge_close_vertices(0.0000001) 
    o3d.visualization.draw_geometries([vox_mesh]) #visualization
    
    #still need to export properly
        
    

In [110]:
simplistic_mesh_creator_voxel(pointCloudMatrix)

In [111]:
import matplotlib.pyplot as plt

In [112]:
def simplistic_mesh_creator_poisson(pointCloudMatrix):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pointCloudMatrix)
    pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()), center=pcd.get_center())
    #o3d.visualization.draw_geometries([pcd])

    pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3))) #remove exiting Normals
    #pcd.estimate_normals() #Estimate new normals base on pointcloud
    radius = 0.1   # 
    max_nn = 30   # 
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn))
    pcd.orient_normals_consistent_tangent_plane(100) #Takes a while but works!
    #pcd.orient_normals_consistent_tangent_plane(100) #Crashes program
    o3d.visualization.draw_geometries([pcd], point_show_normal=True)
    with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
            mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
                pcd, depth=9)
    o3d.visualization.draw_geometries([mesh])
    print('visualize densities')
    densities = np.asarray(densities)
    density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
    density_colors = density_colors[:, :3]
    density_mesh = o3d.geometry.TriangleMesh()
    
    density_mesh.vertices = mesh.vertices
    density_mesh.triangles = mesh.triangles
    density_mesh.triangle_normals = mesh.triangle_normals
    density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
    o3d.visualization.draw_geometries([density_mesh],mesh_show_back_face=True)
    print('remove low density vertices')

    #o3d.visualization.draw_geometries([density_mesh],mesh_show_back_face=True)
    vertices_to_remove = densities < np.quantile(densities, 0.05) #best Density For most cases
    density_mesh.remove_vertices_by_mask(vertices_to_remove)
    print(mesh)
    o3d.visualization.draw_geometries([density_mesh],mesh_show_back_face=True)
   

In [113]:
simplistic_mesh_creator_poisson(pointCloudMatrix)

[Open3D DEBUG] Input Points / Samples: 23065 / 10925
[Open3D DEBUG] #   Got kernel density: 0.115 (s), 612.938 (MB) / 1309.87 (MB) / 4856 (MB)
[Open3D DEBUG] #     Got normal field: 0.046 (s), 622.988 (MB) / 1309.87 (MB) / 4856 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 3.271203e-04 / 7.545031e+00
[Open3D DEBUG] #       Finalized tree: 0.164 (s), 642.801 (MB) / 1309.87 (MB) / 4856 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.258 (s), 629.984 (MB) / 1309.87 (MB) / 4856 (MB)
[Open3D DEBUG] #Set point constraints: 0.039 (s), 628.641 (MB) / 1309.87 (MB) / 4856 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 719314 / 378808 / 443265
[Open3D DEBUG] Memory Usage: 608.672 MB
[Open3D DEBUG] # Linear system solved: 0.432 (s), 633.457 (MB) / 1309.87 (MB) / 4856 (MB)
[Open3D DEBUG] Got average: 0.0120001 (s), 630.566 (MB) / 1309.87 (MB) / 4856 (MB)
[Open3D DEBUG] Iso-Value: 4.976807e-01 = 1.147901e+04 / 2.306500e+04
[Open3D DEBUG] #          Total Solve:       2.5 (s),    130

In [114]:
def generateConvexHull(pointCloudMatrix):

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pointCloudMatrix)
    pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()), center=pcd.get_center())

    hull, _ = pcd.compute_convex_hull()
    hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
    hull_ls.paint_uniform_color((0, 0, 1))
    o3d.visualization.draw_geometries([hull,hull_ls],mesh_show_back_face=True)

In [21]:
generateConvexHull(pointCloudMatrix)